In [1]:
import sys
import random
import numpy as np
import torch
import transformers
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
)
from torch.utils.data import DataLoader

sys.path.append("../")
from pp_utils import (
    get_model_and_tokenizer,
    load_arithmetic_dataloader,
    get_caches,
    compute_topk_components,
    patching_receiver_heads,
    patching_sender_heads,
    get_receiver_layers,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
transformers.set_seed(seed)

/home/local_nikhil/.conda/envs/anima/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, tokenizer = get_model_and_tokenizer(model_name="llama")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.80s/it]


In [8]:
dataloader = load_arithmetic_dataloader(
    tokenizer=tokenizer,
    clean_data_file="../../data/arithmetic_data_clean.json",
    corrupt_data_file="../../data/arithmetic_data_corrupt.json",
    num_samples=10000,
    batch_size=32,
)

In [9]:
correct, total = 0, 0
for data in dataloader:
    input_ids = data["base_tokens"].to(device)
    labels = data["labels"].to(device)

    outputs = model(input_ids=input_ids)
    logits = outputs.logits[:, -1]
    predicted_index = torch.argmax(logits, dim=-1)

    correct += (predicted_index == labels).sum().item()
    total += labels.size(0)

    del input_ids, labels, outputs, logits, predicted_index
    torch.cuda.empty_cache()

print(f"Accuracy: {round(correct/total, 2)}")

Accuracy: 0.5
